In [58]:
import os

In [59]:
%pwd

'e:\\House-Price-Prediction'

In [60]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path
    
    

In [61]:
from mlProject.constants import *
from mlProject.utils.common import read_yaml, create_directories

In [62]:
class ConfigManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):
        
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)
        
        create_directories([self.config.data_root])
        
    def get_data_config(self) -> DataConfig:
        config = self.config.data_file
        
        create_directories([config.root_dir])
        
        data_config = DataConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir
        )
        
        return data_config

In [63]:
import os
import urllib.request as request
import zipfile
from mlProject import logger
from mlProject.utils.common import get_size

In [64]:
class DataIngestion:
    def __init__(self, config: DataConfig):
        self.config = config
        
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(self.config.source_URL, self.config.local_data_file)
            logger.info(f"{filename} downloaded! with following info: \n{headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")
            
    
    def extract_zip_file(self):
        """zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [65]:
"""Pipeline to download and extract the data"""
try:
    config = ConfigManager()
    data_config = config.get_data_config()
    data_ingestion = DataIngestion(config=data_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2024-07-05 12:57:55,374: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-07-05 12:57:55,386: INFO: common: yaml file: params.yaml loaded successfully]
[2024-07-05 12:57:55,386: INFO: common: yaml file: schema.yaml loaded successfully]
[2024-07-05 12:57:55,390: INFO: common: created directory at: data]
[2024-07-05 12:57:55,394: INFO: common: created directory at: data/data_file]
[2024-07-05 12:57:55,394: INFO: 285715339: File already exists of size: ~ 400 KB]
